In [1]:
import numpy as np
from scipy.spatial.distance import cdist

def p_to_minus_half(P):
        # Since P is diagonal, just invert and sqrt the diagonal elements
        diag = np.diag(P)
        inv_sqrt_diag = np.where(diag > 1e-12, diag**-0.5, 0.0)
        return np.diag(inv_sqrt_diag)

In [2]:
np.random.seed(50)  # For reproducibility
N = 4;
M = 7;
d = 2;
beta = 2.1;
x = np.array([[1.0, -2.0], [3.0, 4.5], [-5.0, 6.0], [-1.0, 2.0]])
# np.kron(np.ones((M, 1)), x[0])  # M x d matrix
y = np.array([[1.3,1.9],[1.3,1.9],[2.1,-3.1],[2.1,-3.1],[3.5,6.1],[3.5,6.1],[3.5,6.1]])
# to flatten y we have
Y = y.reshape((M * d, 1))
# Now we make a random M*M diagonal matrix with entries summing  to 1

P =np.zeros((N,M,M*d,M*d))
P_diags = np.zeros((N, M, M ))
for i in range(N):
    for j in range(M):
        first_two = np.ones(2)*np.random.rand();
        second_two = np.ones(2)*np.random.rand();
        third = np.ones(3)*np.random.rand();
        values = np.concatenate((first_two, second_two, third))
        # values += np.random.rand(M)*2  # Add some noise
        values = values / np.sum(values)  # Normalize to sum to 1
        P_diags[i, j, :] = values 
        P_hat_ij = np.diag(values)
        
        P[i,j,:,:] = np.kron(P_hat_ij, np.eye((d)))
       


        

rho = np.ones(N) / N

D = cdist(x, y, "sqeuclidean", out=None)
D_bar = np.sum(np.tile(D[:, None, :], (1, M, 1)) * P_diags, axis=2)
mins = np.min(D_bar, axis=1, keepdims=True)
D_bar_normalized = D_bar - mins
pi_j_i = np.exp(-beta * D_bar_normalized)
pi_j_i /= np.sum(pi_j_i, axis=1, keepdims=True)


P_K = 0.0;

for i in range(N):
    for j in range(M):
        P_A_i_j =  rho[i] * pi_j_i[i, j] * P[i,j]
        P_K += P_A_i_j
Delta = 0.0
for i in range(N):
    z_i = Y - np.kron(np.ones((M, 1)), x[i].reshape((d,1)))
    Z_i = z_i @ z_i.T # shape (M*d, M*d)
    for j in range(M):
        P_A_i_j =  rho[i] * pi_j_i[i, j] * P[i,j]
        
        Delta += p_to_minus_half(P_K)@P_A_i_j@Z_i@P[i,j]@p_to_minus_half(P_K)
cov = 0.0  
for k in range(M):
    for i in range(N):
        z_i = Y - np.kron(np.ones((M, 1)), x[i].reshape((d,1)))
        Z_i = z_i @ z_i.T # shape (M*d, M*d)
        for j in range(M):
            P_A_i_j =  rho[i] * pi_j_i[i, j] * P[i,j]
            cov +=   P_A_i_j * P[i,j] *np.linalg.inv(P_K) @ Z_i

def f1(psi, beta):
    U = (np.eye(M*d) - 2 * beta * Delta)
    # psi is a vector of shape (M*d, 1)
    return psi.T @ (P_K**0.5)@U @ (P_K**0.5)@psi
def constraint(psi):
    term_2 = np.zeros(N)
    for i in range(N):
        out = 0.0
        z_i = Y - np.kron(np.ones((M, 1)), x[i].reshape((d,1)))
        for j in range(M):
            summand = pi_j_i[i,j]*z_i.T @ P[i,j] @ psi
            # print(f"summand i= {i+1} j= {j+1}: {summand}")
            out += summand
        term_2[i] = out[0][0]
    return term_2

In [3]:
np.set_printoptions(precision=5, suppress=True)
print("Delta matrix:")
print(Delta)

Delta matrix:
[[ 4.42408 -2.28859  4.42408 -2.28859  2.05329 -2.71412  2.05329 -2.71412
   2.31781  0.20375  2.31781  0.20375  2.31781  0.20375]
 [-2.28859  2.77531 -2.28859  2.77531 -0.82869  1.27039 -0.82869  1.27039
  -1.05007  1.06596 -1.05007  1.06596 -1.05007  1.06596]
 [ 4.42408 -2.28859  4.42408 -2.28859  2.05329 -2.71412  2.05329 -2.71412
   2.31781  0.20375  2.31781  0.20375  2.31781  0.20375]
 [-2.28859  2.77531 -2.28859  2.77531 -0.82869  1.27039 -0.82869  1.27039
  -1.05007  1.06596 -1.05007  1.06596 -1.05007  1.06596]
 [ 2.05329 -0.82869  2.05329 -0.82869  1.35187 -1.95582  1.35187 -1.95582
   1.18159  0.50788  1.18159  0.50788  1.18159  0.50788]
 [-2.71412  1.27039 -2.71412  1.27039 -1.95582  3.19304 -1.95582  3.19304
  -1.68159 -0.94404 -1.68159 -0.94404 -1.68159 -0.94404]
 [ 2.05329 -0.82869  2.05329 -0.82869  1.35187 -1.95582  1.35187 -1.95582
   1.18159  0.50788  1.18159  0.50788  1.18159  0.50788]
 [-2.71412  1.27039 -2.71412  1.27039 -1.95582  3.19304 -1.95582  3.1

In [4]:
Gamma = Delta[[0,1,4,5,8,9],:]
Gamma = Gamma[:, [0,1,4,5,8,9]]
dummy = np.array([[2.0,2.0,np.sqrt(6.0)], [2.0,2.0,np.sqrt(6.0)], [np.sqrt(6.0),np.sqrt(6.0),3.0]])
dummy_doubled = np.repeat(np.repeat(dummy, 2, axis=0), 2, axis=1)
Gamma = Gamma * dummy_doubled
print("Gamma matrix:")
print(Gamma)

Gamma matrix:
[[ 8.84816 -4.57718  4.10657 -5.42823  5.67745  0.49908]
 [-4.57718  5.55062 -1.65739  2.54078 -2.57213  2.61106]
 [ 4.10657 -1.65739  2.70373 -3.91164  2.8943   1.24406]
 [-5.42823  2.54078 -3.91164  6.38608 -4.11904 -2.31242]
 [ 5.67745 -2.57213  2.8943  -4.11904  4.41462  2.42124]
 [ 0.49908  2.61106  1.24406 -2.31242  2.42124  6.97027]]


In [5]:
eig_val_Delta , eig_vec_Delta = np.linalg.eig(Delta)  # Check eigenvalues of the matrix
for idx, val in enumerate(eig_val_Delta):
    print(f"Eigenvalue {idx}: {val:.2f}")
    print("Eigenvector:", eig_vec_Delta[:, idx])
    print()

Eigenvalue 0: 21.94
Eigenvector: [-0.42593  0.22681 -0.42593  0.22681 -0.23059  0.33759 -0.23059  0.33759
 -0.24635 -0.07734 -0.24635 -0.07734 -0.24635 -0.07734]

Eigenvalue 1: 9.45
Eigenvector: [ 0.11234 -0.36466  0.11234 -0.36466 -0.05091  0.11356 -0.05091  0.11356
 -0.07021 -0.47012 -0.07021 -0.47012 -0.07021 -0.47012]

Eigenvalue 2: 2.01
Eigenvector: [-0.27895  0.05897 -0.27895  0.05897  0.21096 -0.52652  0.21096 -0.52652
 -0.20984 -0.14361 -0.20984 -0.14361 -0.20984 -0.14361]

Eigenvalue 3: 1.38
Eigenvector: [-0.34501 -0.52614 -0.34501 -0.52614 -0.16958 -0.06383 -0.16958 -0.06383
  0.0561   0.2107   0.0561   0.2107   0.0561   0.2107 ]

Eigenvalue 4: 0.09
Eigenvector: [-0.19594 -0.08616 -0.19594 -0.08616  0.60137  0.29386  0.60137  0.29386
  0.06372  0.00773  0.06372  0.00773  0.06372  0.00773]

Eigenvalue 5: 0.00
Eigenvector: [ 0.26594 -0.16689  0.26594 -0.16689  0.09655  0.07413  0.09655  0.07413
 -0.46527  0.20313 -0.46527  0.20313 -0.46527  0.20313]

Eigenvalue 6: 0.00
Eigenvec

In [6]:
eig_val_Gamma , eig_vec_Gamma = np.linalg.eig(Gamma)  # Check eigenvalues of the matrix
for idx, val in enumerate(eig_val_Gamma):
    print(f"Eigenvalue {idx}: {val:.3f}")
    print("Eigenvector:", eig_vec_Gamma[:, idx])
    print()


Eigenvalue 0: 21.944
Eigenvector: [-0.60235  0.32076 -0.32611  0.47742 -0.42669 -0.13396]

Eigenvalue 1: 9.454
Eigenvector: [ 0.15888 -0.51571 -0.07199  0.16059 -0.12161 -0.81427]

Eigenvalue 2: 2.010
Eigenvector: [-0.3945   0.0834   0.29834 -0.74461 -0.36345 -0.24874]

Eigenvalue 3: 1.378
Eigenvector: [ 0.48792  0.74407  0.23983  0.09027 -0.09716 -0.36494]

Eigenvalue 4: 0.000
Eigenvector: [-0.3761   0.23602 -0.13654 -0.10483  0.80587 -0.35183]

Eigenvalue 5: 0.087
Eigenvector: [ 0.2771   0.12185 -0.85047 -0.41558 -0.11037 -0.01339]



In [7]:
def expand_eigenvector(vec6):
    """
    Given a vector of size 6, expand it to size 14 by repeating each block of two values together:
    - The first two values are repeated as a block once (total 4 elements)
    - The second two values are repeated as a block once (total 4 elements)
    - The last two values are repeated as a block twice (total 6 elements)
    Args:
        vec6 (np.ndarray): shape (6,) or (6,1)
    Returns:
        np.ndarray: shape (14,)
    """
    vec6 = np.asarray(vec6).flatten()
    assert vec6.shape[0] == 6, "Input vector must have size 6"
    out = np.concatenate([
        np.tile(vec6[0:2], 2),   # [a, b, a, b]
        np.tile(vec6[2:4], 2),   # [c, d, c, d]
        np.tile(vec6[4:6], 3)    # [e, f, e, f, e, f]
    ])
    power = -1.0
    correction = np.repeat(np.array([np.sqrt(2.0)**power, np.sqrt(2.0)**power, np.sqrt(2.0)**power, np.sqrt(2.0)**power, np.sqrt(3.0)**power, np.sqrt(3.0)**power,np.sqrt(3.0)**power]),2)
    return out
expand_eigenvector(eig_vec_Gamma[:, 0])

array([-0.60235,  0.32076, -0.60235,  0.32076, -0.32611,  0.47742,
       -0.32611,  0.47742, -0.42669, -0.13396, -0.42669, -0.13396,
       -0.42669, -0.13396])

In [8]:
Delta@expand_eigenvector(eig_vec_Gamma[:, 0]).reshape((M * d, 1))/eig_val_Gamma[0]

array([[-0.62785],
       [ 0.32842],
       [-0.62785],
       [ 0.32842],
       [-0.34046],
       [ 0.49859],
       [-0.34046],
       [ 0.49859],
       [-0.36686],
       [-0.12583],
       [-0.36686],
       [-0.12583],
       [-0.36686],
       [-0.12583]])

In [9]:
from scipy.optimize import minimize
beta= 5.15
# Initial guess for psi (shape: (M*d, 1))
psi0 = np.random.rand(M * d, 1)
# psi0 = np.ones((M * d, 1))  # Starting with a zero vector
# Objective function for scipy (expects 1D array)
def obj_fun(psi_flat):
    psi = psi_flat.reshape((M * d, 1))
    return f1(psi, beta).item()

# Constraint function for scipy (expects 1D array, returns 1D array)
def cons_fun(psi_flat):
    psi = psi_flat.reshape((M * d, 1))
    return constraint(psi)

# Norm constraint: ||psi||^2 - 1 = 0
def norm_constraint(psi_flat):
    return np.sum(psi_flat**2) - 1

# Define constraints list for scipy
constraints = [
    {'type': 'eq', 'fun': cons_fun},
    {'type': 'eq', 'fun': norm_constraint}
]
def total_psi(psi_flat):
    candidate = psi_flat.reshape((M * d, 1))
    return f1(candidate, beta) + 2 * beta * np.sum(rho * (constraint(candidate)**2))

# Run the minimizer
result = minimize(obj_fun, psi0.flatten(), constraints=constraints)
# result = minimize(total_psi, psi0.flatten(), constraints=[{'type': 'eq', 'fun': norm_constraint}] )
# Reshape the result to (M*d, 1)
psi_opt = result.x.reshape((M * d, 1))
print("Optimal psi:\n", psi_opt)
print("Objective value:", obj_fun(result.x))
print("Constraint values:", cons_fun(result.x))
print("Norm constraint value:",1- norm_constraint(result.x))
print("Total psi value:", total_psi(result.x)[0,0])

Optimal psi:
 [[-0.14305]
 [-0.08568]
 [-0.14365]
 [-0.0856 ]
 [ 0.6101 ]
 [ 0.30537]
 [ 0.6106 ]
 [ 0.30494]
 [ 0.064  ]
 [ 0.01558]
 [ 0.06367]
 [ 0.01575]
 [ 0.06351]
 [ 0.01585]]
Objective value: -0.0006469485060597163
Constraint values: [ 0.  0. -0. -0.]
Norm constraint value: 0.9999993855186531
Total psi value: -0.0006469485060597163


In [10]:
np.set_printoptions(precision=3, suppress=True)
cons_fun(result.x)

array([ 0.,  0., -0., -0.])

In [11]:
selector = np.array([[1,0,0,0,0,0,0,0,0,0,0,0,0,0],
                     [0,1,0,0,0,0,0,0,0,0,0,0,0,0],
                     [0,0,0,0,1,0,0,0,0,0,0,0,0,0],
                     [0,0,0,0,0,1,0,0,0,0,0,0,0,0],
                     [0,0,0,0,0,0,0,0,0,0,0,0,1,0],
                     [0,0,0,0,0,0,0,0,0,0,0,0,0,1]])
h1h2h3 = selector@psi_opt
h1 = h1h2h3[0:2]
h2 = h1h2h3[2:4]
h3 = h1h2h3[4:6]

h1h1h1 = np.kron(np.ones((3, 1)), h1)  # Repeat h1 twice
h2h2h2 = np.kron(np.ones((3, 1)), h2)  # Repeat h2 twice
h3h3h3 = np.kron(np.ones((3, 1)), h3)  # Repeat h3 twice

v1v2v3 = selector@eig_vec_Delta[:,5]
v1 = v1v2v3[0:2]
v2 = v1v2v3[2:4]
v3 = v1v2v3[4:6]

v1v1v1 = np.kron(np.ones((3, 1)), h1)  # Repeat h1 twice
v2v2v2 = np.kron(np.ones((3, 1)), h2)  # Repeat h2 twice
v3v3v3 = np.kron(np.ones((3, 1)), h3)  # Repeat h3 twice

In [ ]:
# power = -1.0
# correction = np.repeat(np.array([np.sqrt(2.0)**power, np.sqrt(2.0)**power, np.sqrt(2.0)**power, np.sqrt(2.0)**power, np.sqrt(3.0)**power, np.sqrt(3.0)**power,np.sqrt(3.0)**power]),2)
# coeffs = (np.linalg.inv(eig_vec_Delta)@psi_opt)[:,0]
# coeffs

In [12]:

coeffs =(np.linalg.inv(eig_vec_Gamma)@h1h2h3)[:,0]
# coeffs = np.array([-0.01,  0.01,  0.02, -0.02, -0.63,  0.  ])
# coeffs = coeffs/np.min(coeffs)
new_psi = (eig_vec_Gamma @ coeffs).reshape((-1, 1))
new_psi = expand_eigenvector(new_psi).reshape((M * d, 1))
print(coeffs)
print(constraint(new_psi))
print(f1(new_psi, beta).item())                                    

[-0.024  0.006 -0.023  0.028 -0.036 -0.703]
[ 0. -0.  0. -0.]
-0.0006518854296964638


In [ ]:
coeffs =(np.linalg.inv(eig_vec_Delta)@psi_opt)[:,0]
np.sum(coeffs@p_to_minus_half(P_K))

In [ ]:
beta = 1e-6
while beta < 1e6:
    for i in range(3*d):
        candidate = expand_eigenvector(eig_vec_Gamma[:,i]).reshape((M * d, 1))
        if np.linalg.norm(constraint(candidate)) < 1e-6 and f1(candidate, beta)[0,0] < 0.0:
            print("constraint :", constraint(candidate))
            print(f"Eigenvalue:{eig_val_Delta[i]:.2f}", )
            print(f"f1(candidate):{f1(candidate, beta)[0,0]:2f}" )
            total = f1(candidate, beta) + 2* beta * np.sum(rho*(constraint(candidate)**2))

            print(f"Total{i}: {total[0,0]:.2f}")
    beta *=1.5

In [ ]:
beta = 1e-6
while beta < 1e6:
    for i in range(M*d):
        candidate = eig_vec_Delta[:,i].reshape((M * d, 1))
        if np.linalg.norm(constraint(candidate)) < 1e-6 and f1(candidate, beta)[0,0] < 0.0:
            
            print("constraint :", constraint(candidate))
            print(f"Eigenvalue:{eig_val_Delta[i]:.2f}", )
            print(f"f1(candidate):{f1(candidate, beta)[0,0]:2f}" )
            total = f1(candidate, beta) + 2* beta * np.sum(rho*constraint(candidate))
            print(f"Total{i}: {total[0,0]:.2f}")
    beta *=1.5
    

In [ ]:
pi_j_i